In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import joblib


In [21]:
df=pd.read_excel('gym recommendation.xlsx')

In [3]:
df.shape

(14589, 15)

In [4]:
df.sample(7)

,ID,Sex,Age,Height,Weight,Hypertension,Diabetes,BMI,Level,Fitness Goal,Fitness Type,Exercises,Equipment,Diet,Recommendation
7741,7742,Female,43,1.71,90.0,No,No,30.78,Obuse,Weight Loss,Cardio Fitness,"Brisk walking, cycling, swimming, running , or...","Ellipticals, Indoor Rowers,Treadmills, Rowing ...","Vegetables: (Broccoli, Carrots, Spinach, Lettu...",Follow a regular exercise schedule. Adhere to ...
7639,7640,Female,40,1.88,88.7,No,Yes,25.10,Overweight,Weight Loss,Cardio Fitness,"Walking, Yoga, Swimming.","Kettlebell, Dumbbells, Yoga Mat","Vegetables: (Garlic, Mushroom, Green Papper, I...",Follow a regular exercise schedule. Adhere to ...
1263,1264,Female,57,1.69,64.0,Yes,Yes,22.41,Normal,Weight Gain,Muscular Fitness,"Squats, yoga, deadlifts, bench presses, and ov...","Light athletic shoes, resistance bands, light ...","Vegetables: (Garlic, Roma Tomatoes, Capers, Gr...",Follow a regular exercise schedule. Adhere to ...
5042,5043,Male,23,1.60,68.5,Yes,No,26.76,Overweight,Weight Loss,Cardio Fitness,"Brisk walking, cycling, swimming, running , or...","Ellipticals, Indoor Rowers,Treadmills, Rowing ...","Vegetables: (Garlic, Mushroom, Green Papper, I...",Follow a regular exercise schedule. Adhere to ...
5335,5336,Female,32,1.79,87.5,No,Yes,27.31,Overweight,Weight Loss,Cardio Fitness,"Walking, Yoga, Swimming.","Kettlebell, Dumbbells, Yoga Mat","Vegetables: (Garlic, Mushroom, Green Papper, I...",Follow a regular exercise schedule. Adhere to ...
10714,10715,Male,45,1.51,83.5,No,Yes,36.62,Obuse,Weight Loss,Cardio Fitness,"Walking, Yoga, Swimming.","Kettlebell, Dumbbells, Yoga Mat","Vegetables: (Garlic, Mushroom, Green Papper, I...",Follow a regular exercise schedule. Adhere to ...
4803,4804,Male,62,1.64,46.6,No,Yes,17.33,Underweight,Weight Gain,Muscular Fitness,"Squats, yoga, deadlifts, bench presses, and ov...","Dumbbells, barbells and Blood glucose monitor","Vegetables: (Garlic, Roma Tomatoes, Capers and...",Follow a regular exercise schedule. Adhere to ...


In [5]:
df.isnull().sum()

ID                0
Sex               0
Age               0
Height            0
Weight            0
Hypertension      0
Diabetes          0
BMI               0
Level             0
Fitness Goal      0
Fitness Type      0
Exercises         0
Equipment         0
Diet              0
Recommendation    0
dtype: int64

In [6]:
df.columns

Index(['ID', 'Sex', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes',
       'BMI', 'Level', 'Fitness Goal', 'Fitness Type', 'Exercises',
       'Equipment', 'Diet', 'Recommendation'],
      dtype='object')

In [22]:
data=df.drop(['ID','Height','Weight','Level','Fitness Type','Recommendation'],axis=1)

In [23]:
data.head()

,Sex,Age,Hypertension,Diabetes,BMI,Fitness Goal,Exercises,Equipment,Diet
0,Male,18,No,No,16.83,Weight Gain,"Squats, deadlifts, bench presses, and overhead...",Dumbbells and barbells,"Vegetables: (Carrots, Sweet Potato, and Lettuc..."
1,Male,18,Yes,No,16.83,Weight Gain,"Squats, deadlifts, bench presses, and overhead...","Light athletic shoes, resistance bands, and li...","Vegetables: (Tomatoes, Garlic, leafy greens, b..."
2,Male,18,No,Yes,16.83,Weight Gain,"Squats, yoga, deadlifts, bench presses, and ov...","Dumbbells, barbells and Blood glucose monitor","Vegetables: (Garlic, Roma Tomatoes, Capers and..."
3,Male,18,Yes,Yes,16.83,Weight Gain,"Squats, yoga, deadlifts, bench presses, and ov...","Light athletic shoes, resistance bands, light ...","Vegetables: (Garlic, Roma Tomatoes, Capers, Gr..."
4,Male,18,No,No,16.83,Weight Gain,"Squats, deadlifts, bench presses, and overhead...",Dumbbells and barbells,"Vegetables: (Carrots, Sweet Potato, Lettuce); ..."


In [24]:
data.values[0]

array(['Male', 18, 'No', 'No', 16.83, 'Weight Gain',
       'Squats, deadlifts, bench presses, and overhead presses',
       'Dumbbells and barbells',
       'Vegetables: (Carrots, Sweet Potato, and Lettuce); Protein Intake: (Red meats, poultry, fish, eggs, dairy products, legumes, and nuts); Juice: (Fruit juice, watermelon juice, carrot juice, apple juice and mango juice)'],
      dtype=object)

In [25]:
data['Equipment'][10]

'Dumbbells, barbells and Blood glucose monitor'

In [26]:
data['Exercises'][0]

'Squats, deadlifts, bench presses, and overhead presses'

In [29]:
# Encode categorical variables
label_encoders = {}
categorical_cols = ['Sex', 'Hypertension', 'Diabetes', 'Fitness Goal']

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Split features and targets
X = data[['Sex', 'Age', 'Hypertension', 'Diabetes', 'BMI', 'Fitness Goal']]
y_exercises = data['Exercises']
y_equipment = data['Equipment']
y_diet = data['Diet']

# Split into train/test sets
X_train, X_test, y_ex_train, y_ex_test, y_eq_train, y_eq_test, y_d_train, y_d_test = train_test_split(
    X, y_exercises, y_equipment, y_diet, test_size=0.2, random_state=42)

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

# Model for exercises
ex_model = RandomForestClassifier()
ex_model.fit(X_train, y_ex_train)

# Model for equipment
eq_model = RandomForestClassifier()
eq_model.fit(X_train, y_eq_train)

# Model for diet (might need NLP techniques)
diet_model = MultinomialNB()
diet_model.fit(X_train, y_d_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [31]:
from sklearn.metrics import accuracy_score

# Evaluate exercises model
ex_preds = ex_model.predict(X_test)
print("Exercises Accuracy:", accuracy_score(y_ex_test, ex_preds))

# Evaluate equipment model
eq_preds = eq_model.predict(X_test)
print("Equipment Accuracy:", accuracy_score(y_eq_test, eq_preds))

# Evaluate diet model
d_preds = diet_model.predict(X_test)
print("Diet Accuracy:", accuracy_score(y_d_test, d_preds))

Exercises Accuracy: 0.995887594242632
Equipment Accuracy: 0.9211788896504455
Diet Accuracy: 0.6833447566826594


In [34]:
def recommend_workout(user_input):
    # Create DataFrame with proper column names and order
    input_df = pd.DataFrame([user_input], 
                          columns=['Sex', 'Age', 'Hypertension', 'Diabetes', 'BMI', 'Fitness Goal'])
    
    # Encode categorical variables using the same encoders as before
    for col in categorical_cols:
        if col in input_df.columns:  # Check if column exists
            input_df[col] = label_encoders[col].transform(input_df[col])
    
    # Get predictions
    exercises = ex_model.predict(input_df)[0]
    equipment = eq_model.predict(input_df)[0]
    diet = diet_model.predict(input_df)[0]
    
    # Decode the predictions if needed (optional)
    # exercises = label_encoders['Exercises'].inverse_transform([exercises])[0]
    
    return {
        'Exercises': exercises,
        'Equipment': equipment,
        'Diet': diet
    }

# Example usage
user_input = ['Male', 18, 'No', 'No', 16.83, 'Weight Gain']
recommendation = recommend_workout(user_input)
print(recommendation)

{'Exercises': 'Squats, deadlifts, bench presses, and overhead presses', 'Equipment': 'Dumbbells and barbells', 'Diet': np.str_('Vegetables: (Broccoli, Carrots, Spinach, Lettuce, Onion); Protein Intake: (Cheese, Cattoge cheese, Skim Milk, Law fat Milk, and Baru Nuts); Juice: (Fruit Juice, Aloe vera juice, Cold-pressed juice, and Watermelon juice)')}
